In [ ]:
%pylab inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sys 
import os
import pandas as pd
import joblib
#from sklearn.externals import joblib

import re
import numpy as np


In [ ]:
print("Creating Cariac Validation Cohort....")
ca_validation_df = pd.read_csv('ca_validation_group_input (IQVIA validation dataset).csv')
ca_validation_df = ca_validation_df.set_index('patient_id')
out_ca_validation_df = ca_validation_df['codes'].str.split(',',expand = True).stack()
out_ca_validation_df.index = out_ca_validation_df.index.droplevel(-1)
out_ca_validation_df = pd.DataFrame(out_ca_validation_df,columns = ['codes'])
out_ca_validation_df = out_ca_validation_df.reset_index()
out_ca_validation_df['type'] = 2
mask = input("Mask Patient IDs Yes/No?")
if(mask == 1 or mask =='y' or mask =='Y' or mask =='Yes'):
	pat_map = out_ca_validation_df['patient_id'].drop_duplicates().reset_index(drop=True).reset_index()                                         
	pat_map = pat_map.rename(columns = {"index":"new_pat_id"})
	final_ca_validation_df = out_ca_validation_df.merge(pat_map,how='inner',on='patient_id')  
	final_ca_validation_df = final_ca_validation_df.drop('patient_id',axis=1)                                                                 
	final_ca_validation_df = final_ca_validation_df.rename(columns = {"new_pat_id":"patient_id"})
	final_ca_validation_df = final_ca_validation_df[['patient_id','type','codes']]
	final_ca_validation_df.to_csv('ca_validation_model_input.csv',index = False)
else:
	out_ca_validation_df.to_csv('ca_validation_model_input.csv',index = False)
print("Caridac Validation Input Generated Sucessfully")

In [ ]:
os.getcwd()

In [ ]:
# os.chdir(path)
input_df_path = os.getcwd().strip() + '/ca_validation_model_input.csv' #Enter the name of the validation input file
icd_map_path = os.getcwd().strip() + '/icd_map.csv'
ft_imp_path = os.getcwd().strip() + '/feature_importance.csv'
model_file = joblib.load(os.getcwd().strip() + '/rf_best_excl_1_CM_3_6.sav')


In [ ]:
input_df_pre = pd.read_csv(input_df_path)
icd_map_df = pd.read_csv(icd_map_path)
ft_imp_df = pd.read_csv(ft_imp_path)


In [ ]:
ft_list_df = pd.DataFrame(ft_imp_df['feature'])

In [ ]:
#Enter the name of Patient Identifier column
ptid = 'patient_id'

#Enter the name of ICD code version type column
vers_type = 'type'

#Enter the name of the ICD code column
codes = 'codes'

In [ ]:

input_df = input_df_pre


In [ ]:
diagnosis_count = pd.DataFrame({'diag_count' : input_df.groupby('patient_id').count().codes}).reset_index()

In [ ]:
amyloidosis_flag = input_df
amyloidosis_flag['E85_flag'] = np.where(input_df['codes']=='E85','1','0')
amyloidosis_flag['E850_flag'] = np.where(input_df['codes']=='E850','1','0')
amyloidosis_flag['E851_flag'] = np.where(input_df['codes']=='E851','1','0')
amyloidosis_flag['E852_flag'] = np.where(input_df['codes']=='E852','1','0')
amyloidosis_flag['E853_flag'] = np.where(input_df['codes']=='E853','1','0')
amyloidosis_flag['E854_flag'] = np.where(input_df['codes']=='E854','1','0')
amyloidosis_flag['E858_flag'] = np.where(input_df['codes']=='E858','1','0')
amyloidosis_flag['E8581_flag'] = np.where(input_df['codes']=='E8581','1','0')
amyloidosis_flag['E8582_flag'] = np.where(input_df['codes']=='E8582','1','0')
amyloidosis_flag['E8589_flag'] = np.where(input_df['codes']=='E8589','1','0')
amyloidosis_flag['E859_flag'] = np.where(input_df['codes']=='E859','1','0')

In [ ]:
amy_E85 = pd.DataFrame({'max_E85_flag' : amyloidosis_flag.groupby('patient_id')['E85_flag'].max()}).reset_index()
amy_E850 = pd.DataFrame({'max_E850_flag' : amyloidosis_flag.groupby('patient_id')['E850_flag'].max()}).reset_index()
amy_E851 = pd.DataFrame({'max_E851_flag' : amyloidosis_flag.groupby('patient_id')['E851_flag'].max()}).reset_index()
amy_E852 = pd.DataFrame({'max_E852_flag' : amyloidosis_flag.groupby('patient_id')['E852_flag'].max()}).reset_index()
amy_E853 = pd.DataFrame({'max_E853_flag' : amyloidosis_flag.groupby('patient_id')['E853_flag'].max()}).reset_index()
amy_E854 = pd.DataFrame({'max_E854_flag' : amyloidosis_flag.groupby('patient_id')['E854_flag'].max()}).reset_index()
amy_E858 = pd.DataFrame({'max_E858_flag' : amyloidosis_flag.groupby('patient_id')['E858_flag'].max()}).reset_index()
amy_E8581 = pd.DataFrame({'max_E8581_flag' : amyloidosis_flag.groupby('patient_id')['E8581_flag'].max()}).reset_index()
amy_E8582 = pd.DataFrame({'max_E8582_flag' : amyloidosis_flag.groupby('patient_id')['E8582_flag'].max()}).reset_index()
amy_E8589 = pd.DataFrame({'max_E8589_flag' : amyloidosis_flag.groupby('patient_id')['E8589_flag'].max()}).reset_index()
amy_E859 = pd.DataFrame({'max_E859_flag' : amyloidosis_flag.groupby('patient_id')['E859_flag'].max()}).reset_index()
# amy_E85.head()

In [ ]:
x = input_df['patient_id'].nunique()

input_df['ca_flag'] = np.where(input_df['codes']=='E8582','1',np.where(input_df['codes']=='E854','1','0'))
ca_pt = input_df[input_df['ca_flag'] == '1']
ca_pt_id = ca_pt['patient_id']
input_df['hf_flag'] = np.where(input_df['codes']=='I504','1',np.where(input_df['codes']=='I508','1',np.where(input_df['codes']=='I5081','1',np.where(input_df['codes']=='I50','1',np.where(input_df['codes']=='I501','1',np.where(input_df['codes']=='I502','1',np.where(input_df['codes']=='I5020','1',np.where(input_df['codes']=='I5021','1',np.where(input_df['codes']=='I5022','1',np.where(input_df['codes']=='I5023','1',np.where(input_df['codes']=='I503','1',np.where(input_df['codes']=='I5030','1',np.where(input_df['codes']=='I5031','1',np.where(input_df['codes']=='I5032','1',np.where(input_df['codes']=='I5033','1',np.where(input_df['codes']=='I5040','1',np.where(input_df['codes']=='I5041','1',np.where(input_df['codes']=='I5042','1',np.where(input_df['codes']=='I5043','1',np.where(input_df['codes']=='I50810','1',np.where(input_df['codes']=='I50811','1',np.where(input_df['codes']=='I50812','1',np.where(input_df['codes']=='I50813','1',np.where(input_df['codes']=='I50814','1',np.where(input_df['codes']=='I5082','1',np.where(input_df['codes']=='I5083','1',np.where(input_df['codes']=='I5084','1',np.where(input_df['codes']=='I5089','1',np.where(input_df['codes']=='I509','1','0')))))))))))))))))))))))))))))
hf_pt = input_df[input_df['hf_flag'] == '1']
hf_pt_id = hf_pt['patient_id']
df_cd = pd.merge(input_df,ca_pt_id, how='inner', on = 'patient_id')
# df_cd.head(5)
pt_hf_flag = pd.DataFrame({'max_hf_flag' : df_cd.groupby('patient_id')['hf_flag'].max()}).reset_index()
pt_input_hf_flag = pd.DataFrame({'max_hf_flag' : input_df.groupby('patient_id')['hf_flag'].max()}).reset_index()
pt_hf_flag_0 = pt_hf_flag[pt_hf_flag['max_hf_flag']=='0']
total_pat = input_df['patient_id'].nunique()
ca_pat_count = ca_pt_id.nunique()
hf_pat_count = hf_pt_id.nunique()
ca_without_hf = pt_hf_flag_0['patient_id'].nunique()
ca_final_patient_df = pt_hf_flag[pt_hf_flag['max_hf_flag']!='0']
final_patient_df = pt_input_hf_flag[pt_input_hf_flag['max_hf_flag']!='0']
ca_final_pat_id = ca_final_patient_df['patient_id']
final_pat_id= final_patient_df['patient_id']
final_input_df = pd.merge(input_df,final_pat_id, how='inner', on = 'patient_id')

final_input_df = final_input_df[['patient_id','codes','type']]

hf_pt_count = hf_pat_count-(ca_pat_count - ca_without_hf)
ca_pt_count = ca_pat_count - ca_without_hf

In [ ]:
print("A total of " + str(total_pat) + " patients were found. Out of which we discovered " + str(ca_pat_count - ca_without_hf)  + " as Cardiac Amyloid patients and a total of " + str(hf_pat_count-(ca_pat_count - ca_without_hf))  + " Heart Failure Patients.")
if (ca_pat_count - ca_without_hf == 0 ):
    print("Please provide sample data for a few Cardiac Amyloid patients as well as heart failure patients.")
    


In [ ]:
dtype1 = dict(codes = str, type = str)

potential_target_leaks = ['cardiomyopathy_in_diseases_classified_elsewhere','other_forms_of_heart_disease']

def preprocess(df):
    
    def mis_val(df):
        #print ("Count of NA's \n",df.isnull().sum())
        nans = lambda df: df[df.isnull().any(axis=1)]
        #print ("\n Rows with NA's \r\n", nans(df))
        df_wo_na = df.dropna()
        return df_wo_na
    
    df_cln = mis_val(df)
    
    def icd_fix(icd_version):
        conditions =  (icd_version.str.contains('10'))| (icd_version == '2'),(icd_version.str.contains('9'))| (icd_version == '1')
        choices = ['ICD10', 'ICD9']
        icd_fix = np.select(conditions, choices, default='NA')
        return icd_fix
    
    df_cln['type_fix'] = icd_fix(df_cln[vers_type])
    
    df_map = pd.merge(df_cln,icd_map_df,left_on=["type_fix",codes],right_on=["type","codes"] ,how = 'left')
    
    df_map_cln = mis_val(df_map)
        
    keys = ['short_desc','major','sub_chapter']

    df_feature = df_map_cln.melt(id_vars= ptid,value_vars=keys,var_name= 'source',value_name='feature') #different syntax in lower versions
    
    features = pd.DataFrame(df_feature['feature'].drop_duplicates())
    
    features['feature_cln'] = features['feature'].str.lower().str[:100].replace(to_replace="[^A-Za-z0-9]", value="_", regex=True)
    
    df_feature = df_feature.merge(features,on = 'feature',how='inner')
    #df_feature['feature_cln'] = df_feature['feature'].str.lower().str[:100].replace(to_replace="[^A-Za-z0-9]", value="_", regex=True)
    
    df_feature = df_feature.drop(['feature'],axis=1)
    
    df_feature_fitler = pd.merge(ft_list_df,df_feature,left_on = 'feature',right_on= 'feature_cln',how= 'left')
    
    df_feature_cols = df_feature_fitler[[ptid,'feature']].copy()
    
    df_feature_cols['presence_flag'] = '1'
    
    df_ft_pivot = df_feature_cols.pivot_table(index=ptid, columns='feature', values='presence_flag', aggfunc=np.max,dropna = False,fill_value = '0')

    #df_pivot_no_leaks = df_ft_pivot.drop(potential_target_leaks, axis = 1) 
    
    return df_ft_pivot
    
        

In [ ]:
final_df = preprocess(final_input_df.astype(dtype1))

In [ ]:
# Prediction

target_map = {u'1': 1, u'0': 0}

rf_random_predictions = model_file.predict(final_df)
rf_random_probas = model_file.predict_proba(final_df)

rf_random_predictions = pd.Series(data=rf_random_predictions, index=final_df.index, name='predicted_value')
cols = [
    u'probability_of_value_%s' % label
    for (_, label) in sorted([(int(target_map[label]), label) for label in target_map])
]

rf_random_probabilities = pd.DataFrame(data=rf_random_probas, index=final_df.index, columns=cols)

results_test_rf_random = pd.concat([rf_random_predictions,rf_random_probabilities],axis=1)

results_test_rf_random.reset_index(level=0, inplace=True)
#Prediction output



In [ ]:
flag_pt = pd.DataFrame()
final_input_df['cohort_flag'] = np.where(final_input_df["codes"]== 'E8582' , '1',np.where(final_input_df["codes"]== 'E854' , '1','0'))
flag = final_input_df[final_input_df['cohort_flag'] == '1']
flag_pt['patient_id'] = flag['patient_id']
flag_pt =flag_pt.drop_duplicates()
flag_pt['cohort_flag']  = '1'


In [ ]:
df_cd = pd.merge(flag_pt, results_test_rf_random, how='outer', on = 'patient_id')
final = df_cd[['patient_id','cohort_flag','predicted_value','probability_of_value_0','probability_of_value_1']]

final['cohort_f'] = np.where(final["cohort_flag"]== '1', 1,0)
final_cohort = final
final = final.fillna(0)

final = pd.merge(final, diagnosis_count, how='left', on = 'patient_id')

final = pd.merge(final, amy_E85, how='left', on = 'patient_id')
final = pd.merge(final, amy_E850, how='left', on = 'patient_id')
final = pd.merge(final, amy_E851, how='left', on = 'patient_id')
final = pd.merge(final, amy_E852, how='left', on = 'patient_id')
final = pd.merge(final, amy_E853, how='left', on = 'patient_id')
final = pd.merge(final, amy_E854, how='left', on = 'patient_id')
final = pd.merge(final, amy_E858, how='left', on = 'patient_id')
final = pd.merge(final, amy_E8581, how='left', on = 'patient_id')
final = pd.merge(final, amy_E8582, how='left', on = 'patient_id')
final = pd.merge(final, amy_E8589, how='left', on = 'patient_id')
final = pd.merge(final, amy_E859, how='left', on = 'patient_id')


In [ ]:
y_test = final['cohort_f']

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
preds=final['probability_of_value_1']

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
#probs = model.predict_proba(X_test)
#preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# print ('ROC AUC'),print(roc_auc),print('')
print ('Accuracy:'),print(metrics.accuracy_score(y_test, final['predicted_value']))
print ('\nF1 Score:'),print(metrics.f1_score(y_test, final['predicted_value']))
print ('\nRecall:'),print(metrics.recall_score(y_test, final['predicted_value']))
print ('\nPrecision:'),print(metrics.precision_score(y_test, final['predicted_value']))
print ('\nClassification report:'),print(metrics.classification_report(y_test, final['predicted_value']))
print ('\nConfusion matrix:'),print(metrics.confusion_matrix(y_test, final['predicted_value']))

In [ ]:
accuracy = metrics.accuracy_score(y_test, final['predicted_value'])
f1_score = metrics.f1_score(y_test, final['predicted_value'])
recall = metrics.recall_score(y_test, final['predicted_value'])
precision = metrics.precision_score(y_test, final['predicted_value'])
confusion_matrix = metrics.confusion_matrix(y_test, final['predicted_value'])


In [ ]:
true_positive = confusion_matrix[1][1]
true_negative = confusion_matrix[0][0]
false_positive = confusion_matrix[0][1]
false_negative = confusion_matrix[1][0]

In [ ]:
matrix = pd.DataFrame()
matrix.loc[0,1]='Accuracy:'
matrix.loc[0,2]= accuracy
matrix.loc[1,1]='F1 Score:'
matrix.loc[1,2]= f1_score
matrix.loc[2,1]='Recall:'
matrix.loc[2,2]= recall
matrix.loc[3,1]='Precision:'
matrix.loc[3,2]= precision
matrix.loc[4,1]='ROC AUC:'
matrix.loc[4,2]= roc_auc
matrix.loc[5,1]= ""
matrix.loc[5,2]= ""


In [ ]:
matrix.columns=[['metric','value']]
matrix = matrix.reset_index()


In [ ]:
matrix_2 = pd.DataFrame()
matrix_2.loc[0,0] = "Confusion"
matrix_2.loc[0,1] = "Matrix"
matrix_2.loc[0,2] = "Predicted"
matrix_2.loc[0,3] = "Predicted"

matrix_2.loc[1,2] = 1
matrix_2.loc[1,3] = 0
matrix_2.loc[2,0] = "Actual"
matrix_2.loc[2,1] = 1
matrix_2.loc[3,0] = "Actual"
matrix_2.loc[3,1] = 0

# matrix_2
cols=[0,1,2,3]
matrix_2 = matrix_2[cols]
matrix_2.loc[2,2] = true_positive
matrix_2.loc[3,3] = true_negative
matrix_2.loc[3,2] = false_positive
matrix_2.loc[2,3] = false_negative
matrix_2.loc[4,0] = ""
matrix_2.loc[4,1] = ""
matrix_2.loc[4,2] = ""
matrix_2.loc[4,3] = ""

In [ ]:
cols=[0,1,2,3]
matrix_2 = matrix_2[cols]




In [ ]:
final['patient_id'] = final.reset_index().index
final.shape

final_df = final[['patient_id', 'cohort_flag', 'probability_of_value_1','predicted_value','diag_count','max_E85_flag','max_E850_flag','max_E851_flag','max_E852_flag','max_E853_flag','max_E854_flag','max_E858_flag','max_E8581_flag','max_E8582_flag','max_E8589_flag','max_E859_flag',]]
final_df['type'] = np.where(final_df['cohort_flag']== '1',np.where(final_df['predicted_value']=='1',"True Positive","True Negative"),np.where(final_df['predicted_value']=='1',"False Positive","False Negative"))



In [ ]:
output_path = os.getcwd().strip()+'\\Validation Outputs'

In [ ]:
try :
    os.chdir(output_path)
except:
    os.mkdir('Validation Outputs')
    os.chdir(output_path)
pd.DataFrame(matrix_2).to_csv('Model metrics.csv', header = False, index  = False)
pd.DataFrame(matrix).to_csv('Model metrics.csv', mode='a', header=False, index = False)
pd.DataFrame(final_df).to_csv('final_predictions.csv')
# pd.DataFrame(final_merged_patient_count_list_df).to_csv('Phenotype combination comparison.csv')
